# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
!pip install pyautogen[gemini,retrievechat,lmm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
# llm_config = {"model": "gpt-3.5-turbo"}

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

llm_config = {
        "model": "gemini-pro",
        "api_key": GEMINI_API_KEY, #"your Google's GenAI Key goes here",
        "api_type": "google"
    }

## The task!

In [3]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

{'content': '**DeepLearning.AI: Unlocking the Power of Deep Learning**\n\nDeepLearning.AI is a leading online learning platform that empowers individuals with deep learning knowledge and skills. Founded by Andrew Ng, a pioneer in artificial intelligence, it offers comprehensive courses that guide learners from fundamentals to cutting-edge applications.\n\nThrough interactive exercises, hands-on projects, and expert-led discussions, DeepLearning.AI unlocks the potential of deep learning for various industries, including healthcare, finance, and autonomous systems. By fostering collaboration and a global community of learners, it inspires individuals to create transformative technology solutions.', 'role': 'assistant', 'function_call': None, 'tool_calls': None}


## Adding reflection

Create a critic agent to reflect on the work of the writer agent.

In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**DeepLearning.AI: Unlocking the Power of Deep Learning**

DeepLearning.AI is a leading online learning platform that empowers individuals with deep learning knowledge and skills. Founded by Andrew Ng, a pioneer in artificial intelligence, it offers comprehensive courses that guide learners from fundamentals to cutting-edge applications.

Through interactive exercises, hands-on projects, and expert-led discussions, DeepLearning.AI unlocks the potential of deep learning for various industries, including healthcare, finance, and autonomous systems. By fostering collaboration and a global community of learners, it inspires individuals to create transformative technology solutions.

----------------------------------------------------

## Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**DeepLearning.AI: Unlocking the Power of Deep Learning**

DeepLearning.AI is a leading online learning platform that empowers individuals with deep learning knowledge and skills. Founded by Andrew Ng, a pioneer in artificial intelligence, it offers comprehensive courses that guide learners from fundamentals to cutting-edge applications.

Through interactive exercises, hands-on projects, and expert-led discussions, DeepLearning.AI unlocks the potential of deep learning for various industries, including healthcare, finance, and autonomous systems. By fostering collaboration and a global community of learners, it inspires individuals to create transformative technology solutions.

----------------------------------------------------

TypeError: sequence item 0: expected str instance, dict found

## Get the summary

In [18]:
print(res.summary)

**Refined Blog Post**

**DeepLearning.AI: Empowering Learners with Deep Learning Expertise**

DeepLearning.AI, led by AI pioneer Andrew Ng, is an esteemed online learning platform that unlocks the transformative power of deep learning. Its comprehensive courses guide learners from foundational concepts to cutting-edge applications, empowering them to make a meaningful impact across industries.

Through interactive exercises, hands-on projects, and expert-led discussions, DeepLearning.AI fosters a collaborative learning environment. By connecting learners with a global community of practitioners, it inspires innovation and the creation of groundbreaking technology solutions.

To further enhance the learning experience, DeepLearning.AI is committed to providing increased opportunities for learner interaction. Discussion forums and group projects are being explored to facilitate knowledge sharing and foster collaboration.

DeepLearning.AI remains an invaluable resource for aspiring deep l